In [16]:
import json
import pandas as pd
import string
import time
import numpy as ny

In [2]:
df=pd.read_csv(r'C:\Users\rejee\Desktop\Agitation.csv')

In [10]:
df.head(10)
df = df.drop(columns=['timestamp (ET)', 'agitimestamp (ET)','Location','Behaviour'])
#df1 = df.rename(columns = {'overall': 'rating', 'asin': 'productID'}, inplace = False)

KeyError: "['timestamp (ET)', 'agitimestamp (ET)', 'Location', 'Behaviour'] not found in axis"

In [4]:
df.head(10)

,level,Observation
0,5,Withdrawn
1,3,"Vocal1, Withdrawn"
2,3,Withdrawn
3,4,Withdrawn
4,4,"Vocal2, Withdrawn"
5,3,Withdrawn
6,4,"Communication, Vocal1, Withdrawn"
7,3,"Vocal1,0"
8,9,Vocal1; Vocal2;
9,7,Repetition; Vocal1; Vocal2;


In [12]:
df.drop_duplicates(subset=['level','Observation'],keep='first',inplace=True)

In [5]:
def clean_text(text ): 
    delete_dict = {sp_character: ' ' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))]) 
    
    return text2.lower().split(' ')

In [20]:
df2 = df.sample(n=20)
df2['Observation']= df2['Observation'].apply(clean_text)

In [21]:
sentences = df2['Observation'].tolist()

In [24]:
print(len(sentences))
print(sentences[1])
print(sentences[2])

20
['ambulation', 'clothing', 'frustration', 'vocal1', 'withdrawn']
['ambulation', 'frustration', 'repetition']


In [25]:
import gensim

In [26]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

In [54]:
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 5

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        elif self.epoch % 100 == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        
        
        self.epoch += 1
        self.loss_previous_step = loss

In [55]:
w2v_model = Word2Vec(vector_size  = 100,
                     window = 15,
                     min_count = 2,
                     workers = 20,
                     sg = 1,
                     negative = 5,
                     sample = 1e-5)
# build vovab


w2v_model.build_vocab(sentences)

In [56]:
start = time.time()
w2v_model.train(sentences, 
                total_examples=w2v_model.corpus_count, 
                epochs=1001, 
                report_delay=1,
                compute_loss = True, # set compute_loss = True
                callbacks=[callback()]) # add the callback class
end = time.time()


print("elapsedtime in seconds :"+ str(end - start))
w2v_model.save(r'C:\Users\rejee\Desktop\Agitation.csv')

Loss after epoch 100: 0.0
Loss after epoch 200: 0.0
Loss after epoch 300: 0.0
Loss after epoch 400: 0.0
Loss after epoch 500: 0.0
Loss after epoch 600: 0.0
Loss after epoch 700: 0.0
Loss after epoch 800: 0.0
Loss after epoch 900: 0.0
Loss after epoch 1000: 0.0
elapsedtime in seconds :10.558250904083252


In [ ]:
w2v_model.wv.

In [57]:
reloaded_w2v_model = Word2Vec.load(r'C:\Users\rejee\Desktop\Agitation.csv')
words = list(reloaded_w2v_model.wv.vocab)
print('Vocab size: '+str(len(words)))
w1 = 'Withdrawn'
print("Top 3 words similar to Withdrawn:",\
      reloaded_w2v_model.wv.most_similar(positive = w1,topn =3))
w1 = 'Vocal1'
print("Top 3 words similar to Vocal1:",\
      reloaded_w2v_model.wv.most_similar(positive = w1,topn =3))
print("Similarity between Communication and Repetition:"+\
      str(reloaded_w2v_model.wv.similarity(w1="Clothing",w2="Vocal2")))
print("Similarity between  Vocal1 and Vocal2:"+\
      str(reloaded_w2v_model.wv.similarity(w1="Vocal1;",w2="Vocal2")))
words = list(reloaded_w2v_model.wv.vocab)
print('Vocab size: '+str(len(words)))
w1 = 'Withdrawn'
print("Top 3 words similar to Withdrawn:",\
      reloaded_w2v_model.wv.most_similar(positive = w1,topn =3))
w1 = 'Vocal1'
print("Top 3 words similar to Vocal1:",\
      reloaded_w2v_model.wv.most_similar(positive = w1,topn =3))
print("Similarity between Communication and Repetition:"+\
      str(reloaded_w2v_model.wv.similarity(w1="earphones",w2="headphones")))
print("Similarity between Vocal1 and Vocal2:"+\
      str(reloaded_w2v_model.wv.similarity(w1="Vocal1",w2="Vocal2")))

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

In [ ]:
# Set the parameters for SVM
svm_params = {'kernel': ['linear', 'poly', 'rbf'], 'C': [0.1, 1, 10]}

# Set the parameters for logistic regression
logreg_params = {'penalty': ['l1', 'l2'], 'C': [0.1, 1, 10]}

# Set the parameters for LSTM
lstm_params = {'num_lstm_layers': [1, 2], 'lstm_layer_size': [32, 64, 128]}

# Create a parameter grid for SVM
svm_param_grid = dict(svm_params)

# Create a parameter grid for logistic regression
logreg_param_grid = dict(logreg_params)

# Create a parameter grid for LSTM
lstm_param_grid = dict(lstm_params)

# Define the models to use in the grid search
models = [('svm', SVC(), svm_param_grid),
          ('logreg', LogisticRegression(), logreg_param_grid),
          ('lstm', Sequential(), lstm_param_grid)]